In [1]:
import os
import sys
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir,".."))
project_root = os.path.abspath(os.path.join(project_root,"..")) # one level more down
paths_to_add = [project_root,
                os.path.join(project_root,"src","init")]
for path in paths_to_add:
    if path not in sys.path:
        sys.path.append(path)

In [2]:
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY
# connection to db
connection_string = os.getenv('CONNECTION_STRING_DB')

In [3]:
# Imports

import agent
from langchain_core.messages import AIMessage, HumanMessage

graph = agent.graph

orchestrator = agent.orchestrator
run_control_flow = agent.run_control_flow
generate_answer = agent.generate_answer
create_sql_query_or_queries = agent.create_sql_query_or_queries
extract_analytical_intent = agent.extract_analytical_intent

# Import initialization components
from src.init.initialization import (
    llm, llm_fast, create_config, tracer,
    objects_documentation, sql_dialect, connection_string
)

question = 'placeholder'
test_state = {
'objects_documentation':objects_documentation,
'sql_dialect': sql_dialect,
'messages_log':[],
'intermediate_steps' : [],
'analytical_intent': [],
'current_question':question,
'current_sql_queries': [],
'generate_answer_details': {
    'key_assumptions': [],
    'agent_questions': [],
    'ambiguity_explanation': ''
},
'llm_answer': AIMessage(content=''),
'scenario': ''
,'search_terms_output':[]
}

✅ All terms in synonyms exist in key_terms


In [ ]:
search_terms = agent.search_terms
key_terms = agent.key_terms
synonyms = agent.synonyms
related_terms = agent.related_terms

# User asks for synonyms
user_question = 'aum by clients'
search_terms_output = search_terms(user_question, key_terms, synonyms, related_terms)
search_terms_output

In [ ]:
# start the conversation with the graph
question = 'liquid assets by practice segments'

test_state['current_question'] = question
vector_store = None  # reset vector store
config, thread_id = create_config('Run Agent',True)

result = graph.invoke(test_state, config = config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer'].content}'''
print(display)

In [ ]:
# continue the conversation with the graph (followup 1)
question = 'can you say again which one is the best one? sorry i missed it'

test_state['current_question'] = question
vector_store = None  # reset vector store
config, _ = create_config('Run Agent', False, thread_id) # (re-use same thread)
result = graph.invoke(test_state, config)
display = f'''Analytical intent: {result['analytical_intent']}\n\nSQL query: {result['current_sql_queries']}\n\nGenerate Answer Details: {result['generate_answer_details']}\n\nAnswer: {result['llm_answer']}'''
print(display)

In [21]:
import agent
from langchain_core.messages import AIMessage, HumanMessage

orchestrator = agent.orchestrator
run_control_flow = agent.run_control_flow
generate_answer = agent.generate_answer
create_sql_query_or_queries = agent.create_sql_query_or_queries
extract_analytical_intent = agent.extract_analytical_intent
reset_state = agent.reset_state

# Import initialization components
from src.init.initialization import  llm, llm_fast, create_config, tracer, objects_documentation, sql_dialect, connection_string

question = 'aum by clients'

test_state = {
'objects_documentation':objects_documentation,
'sql_dialect': sql_dialect,
'messages_log':[],
'intermediate_steps' : [],
'analytical_intent': [],
'current_question':question,
'current_sql_queries': [],
'generate_answer_details': {
    'key_assumptions': [],
    'agent_questions': [],
    'ambiguity_explanation': ''
},
'llm_answer': AIMessage(content=''),
'scenario': ''
,'search_terms_output':[]
}

#test_state = reset_state(test_state)
#orchestrator(test_state)
#test_state = run_control_flow(test_state) # extract_analytical_intent
#test_state = run_control_flow(test_state) # create sql query + execute sql query
#orchestrator(test_state)
#test_state = run_control_flow(test_state) # generate answer + manage memory
# test_state = generate_answer.invoke({'state':test_state})

In [ ]:
# run reset_state
test_state = reset_state(test_state)
print(test_state['objects_documentation']) # check objects_documentation
print(test_state['scenario']) # scenario

In [23]:
# run orchestrator
orchestrator(test_state)
print(test_state['analytical_intent']) # analytical intent
print(test_state['scenario']) # scenario
print(test_state['intermediate_steps'][-1].tool) # next action

[]

extract_analytical_intent


In [ ]:
# run extract analytical intent
test_state = run_control_flow(test_state)
print(test_state['analytical_intent']) # analytical intent
print(test_state['scenario']) # scenario
print(test_state['intermediate_steps'][-1].tool) # next action
print(test_state['search_terms_output'])
print(test_state['generate_answer_details'])

In [ ]:
# run create sql query
test_state = run_control_flow(test_state)
print(test_state['analytical_intent']) # analytical intent
print(test_state['current_sql_queries']) # scenario
print(test_state['generate_answer_details']['key_assumptions'])
print(test_state['intermediate_steps'][-1].tool) # next action

In [25]:
from agent import extract_terms_used_in_intent, add_key_assumptions_from_analytical_intent

analytical_intent = test_state['analytical_intent']
search_terms_output = test_state['search_terms_output']
# extract_terms_used_in_intent
add_key_assumptions_from_analytical_intent(analytical_intent, search_terms_output)

{'key_assumptions': []}

In [8]:
import agent
from langchain_core.messages import AIMessage, HumanMessage

orchestrator = agent.orchestrator
run_control_flow = agent.run_control_flow
generate_answer = agent.generate_answer
create_sql_query_or_queries = agent.create_sql_query_or_queries
extract_analytical_intent = agent.extract_analytical_intent
reset_state = agent.reset_state
search_terms = agent.search_terms
key_terms = agent.key_terms
synonyms = agent.synonyms
related_terms = agent.related_terms

# Import initialization components
from src.init.initialization import  llm, llm_fast, create_config, tracer, objects_documentation, sql_dialect, connection_string

question = 'aum by clients'
sql_dialect = 'PostgreSQL'
query = ''' SELECT fam.household_key, SUM(fam.account_assets) AS account_assets FROM public.fact_account_monthly fam INNER JOIN public.household h ON fam.household_key = h.household_key WHERE fam.snapshot_date = '2025-09-30' AND h.household_status = 'Active' AND h.to_date = '9999-12-31' GROUP BY fam.household_key'''
analytical_intent = '''["Calculate account_assets from fact_account_monthly grouped by household_key for snapshot_date = '2025-09-30', filtering for household.household_status = 'Active' and household.to_date = '9999-12-31'"]'''
search_terms_output = search_terms(question, key_terms, synonyms, related_terms)

objects_documentation = '''
Table public.date: Calendar dimension table for time-based analysis
Columns:
  - Column calendar_day: Specific calendar date (PRIMARY KEY)
  - Column month_name: Full month name (January, February, etc.)
  - Column month: Month number (1-12)
  - Column day_of_month: Day within the month (1-31)
  - Column month_start_date: First day of the month
  - Column month_end_date: Last day of the month
  - Column quarter: Quarter number (1-4)
  - Column quarter_name: Quarter label (Q1, Q2, Q3, Q4)
  - Column quarter_start_date: First day of the quarter
  - Column quarter_end_date: Last day of the quarter
  - Column year: Four-digit year
  - Column is_weekend: Boolean flag indicating weekend days

Table public.business_line: Investment product service lines offered by Capital Partners to advisors
Columns:
  - Column business_line_key: Unique identifier for Capital Partners business line (PRIMARY KEY)
  - Column business_line_name: Capital Partners product service category (Managed Portfolio, SMA, Mutual Fund Wrap, Annuity, Cash)
    Values in column business_line_name: Annuity | Cash | Managed Portfolio | Mutual Fund Wrap | Separately Managed Account

Table public.advisors: Financial advisors with historical tracking (SCD Type 2)
Columns:
  - Column advisor_key: Unique surrogate key (PRIMARY KEY)
  - Column advisor_id: Natural business identifier for the advisor
  - Column advisor_tenure: Years of experience as an advisor (1-40 years)
  - Column firm_name: Name of the firm the advisor is part of (NOT Capital Partners)
  - Column firm_affiliation_model: Business model of the advisor firm (RIA, Hybrid RIA, Broker-Dealer, etc.)
    Values in column firm_affiliation_model: Bank/Trust | Broker-Dealer W-2 | Bank/Trust | Broker-Dealer W-2 | Hybrid RIA | Independent BD | Insurance BD | RIA | Wirehouse
  - Column advisor_role: Position within the firm (Lead Advisor, Associate, etc.)
    Values in column advisor_role: Associate Advisor | Client Service Associate | Lead Advisor | Portfolio Manager | Relationship Manager
  - Column advisor_status: Current employment status (Active/Terminated)
    Values in column advisor_status: Active | Terminated
  - Column practice_segment: Size classification of advisory practice
    Values in column practice_segment: Ensemble | Enterprise | Small Team | Solo Practice
  - Column from_date: Effective start date for this record version
  - Column to_date: Effective end date for this record version

Table public.household: Client households with historical tracking (SCD Type 2)
Columns:
  - Column household_key: Unique surrogate key (PRIMARY KEY)
  - Column household_id: Natural business identifier for the household
  - Column household_tenure: Years as a client (1-40 years)
  - Column household_registration_type: Legal registration structure (Individual, Joint, Trust, Institutional)
    Values in column household_registration_type: Individual | Institutional | Joint | Trust
  - Column household_registration_date: Date when household became a client
  - Column household_segment: Client service model classification
    Values in column household_segment: Active Trader | Advice-Seeking | Business/Institutional | Discretionary Managed | Retirement Income
  - Column household_status: Current relationship status (Active/Terminated)
    Values in column household_status: Active | Terminated
  - Column household_advisor_id: ID of the primary advisor serving this household
  - Column from_date: Effective start date for this record version
  - Column to_date: Effective end date for this record version

Table public.account: Individual investment accounts with historical tracking (SCD Type 2)
Columns:
  - Column account_key: Unique surrogate key (PRIMARY KEY)
  - Column account_id: Natural business identifier for the account
  - Column advisor_key: Foreign key to advisors table
  - Column household_key: Foreign key to household table
  - Column business_line_key: Foreign key to business_line table
  - Column account_type: Tax classification (Taxable, IRA, 401k, Trust, Custody)
    Values in column account_type: 401k | Custody | IRA | Taxable | Trust
  - Column account_custodian: Firm holding the assets (Schwab, Fidelity, Pershing, etc.)
    Values in column account_custodian: BankTrust | Fidelity | In-House | Pershing | Schwab
  - Column opened_date: Date the account was opened
  - Column account_status: Current status (Open/Closed)
    Values in column account_status: Closed | Open
  - Column closed_date: Date account was closed (if applicable)
  - Column account_risk_profile: Investment risk tolerance (Conservative, Moderate, Aggressive)
    Values in column account_risk_profile: Aggressive | Conservative | Moderate
  - Column from_date: Effective start date for this record version
  - Column to_date: Effective end date for this record version

Table public.product: Investment products and securities
Columns:
  - Column product_id: Unique identifier for investment product (PRIMARY KEY)
  - Column asset_category: High-level asset classification (Equity, Fixed Income, Multi-Asset, Cash)
    Values in column asset_category: Cash | Equity | Fixed Income | Multi-Asset
  - Column asset_subcategory: Detailed product classification within asset category
    Values in column asset_subcategory: Balanced Fund (60/40) | Common Stock | Equity Mutual Fund| Investment-Grade Corporate Bond | Money Market Fund | Municipal Bond | Preferred Stock | Target-Date Fund | U.S. Treasury Bill | U.S. Treasury Note
  - Column product_line: Distribution channel or wrapper type
    Values in column product_line: Annuity Contract | ETF | Money Market | Mutual Fund | Separately Managed Account Strategy
  - Column product_name: Full name of the investment product

Table public.tier_fee: Fee schedule based on asset levels by business line
Columns:
  - Column tier_fee_id: Unique identifier (PRIMARY KEY)
  - Column business_line_key: Foreign key to business_line table
  - Column tier_min_aum: Minimum assets under management for this fee tier
  - Column tier_max_aum: Maximum assets under management for this fee tier
  - Column tier_fee_bps: Fee rate in basis points (e.g., 90 = 0.90%). Non-additive measure.

Table public.advisor_payout_rate: Commission/payout rates by firm affiliation model
Columns:
  - Column firm_affiliation_model: Type of firm affiliation (PRIMARY KEY). Source is table advisors, column firm_affiliation_model.
    Values in column firm_affiliation_model: Bank/Trust | Broker-Dealer W-2 | Hybrid RIA | Independent BD | Insurance BD | RIA | Wirehouse
  - Column advisor_payout_rate: Percentage of revenue paid to advisor (e.g., 0.7800 = 78%). Non-additive measure.

Table public.fact_account_monthly: Monthly account performance and asset data
Columns:
  - Column snapshot_date: End-of-month date for the data snapshot
  - Column account_key: Foreign key to account table
  - Column account_monthly_return: Monthly investment return percentage. Non-additive measure.
  - Column account_net_flow: Net deposits/withdrawals during the month. Additive measure.
  - Column account_assets_previous_month: Asset value at start of month. Semi-additive measure.
  - Column account_assets: Asset value at end of month. Semi-additive measure.
  - Column advisor_key: Foreign key to advisors table (current advisor)
  - Column household_key: Foreign key to household table
  - Column business_line_key: Foreign key to business_line table

Table public.fact_account_product_monthly: Monthly asset allocation by product for each account
Columns:
  - Column snapshot_date: End-of-month date for the allocation snapshot
  - Column account_key: Foreign key to account table
  - Column product_id: Foreign key to product table
  - Column product_allocation_pct: Percentage of account allocated to this product. Non-additive measure.

Table public.fact_household_monthly: Monthly aggregated household data. This table aggregates fact_account_monthly at household level.
Columns:
  - Column snapshot_date: End-of-month date for the data snapshot
  - Column household_key: Foreign key to household table
  - Column household_assets: Total assets across all household accounts. Aggregation over table fact_account_monthly -> column account_assets. Semi-additive measure.
  - Column asset_range_bucket: Categorized asset range for segmentation
    Values in column asset_range_bucket: $0 – $100k | $100k – $250k | $1M – $5M | $250k – $500k | $500k – $1M
  - Column high_net_worth_flag: Boolean indicator for high-net-worth status
  - Column household_net_flow: Net deposits/withdrawals across all accounts. Aggregation of table fact_account_monthly -> column account_net_flow. Additive measure.

Table public.fact_revenue_monthly: Monthly fee and revenue calculations
Columns:
  - Column snapshot_date: End-of-month date for revenue calculation
  - Column account_key: Foreign key to account table
  - Column advisor_key: Foreign key to advisors table
  - Column household_key: Foreign key to household table
  - Column business_line_key: Foreign key to business_line table
  - Column account_assets: Asset value used for fee calculation. Semi-additive measure.
  - Column fee_percentage: Annual fee rate applied to assets. Non-additive measure.
  - Column gross_fee_amount: Total fee charged before deductions. Equals to account_assets x fee_percentage. Additive measure.
  - Column third_party_fee: Fees paid to external parties. Additive measure.
  - Column advisor_payout_rate: Percentage of net revenue paid to advisor. Source is table advisor_payout_rate -> column advisor_payout_rate. Non-additive measure.
  - Column advisor_payout_amount: Dollar amount paid to advisor. Equals to (gross_fee_amount - third_party_fee) x advisor_payout_rate. Additive measure.
  - Column net_revenue: Revenue retained by Capital Partners. Equals to gross_fee_amount - third_party_fee - advisor_payout_amount. Additive measure.

Table public.fact_customer_feedback: Client satisfaction and feedback data
Columns:
  - Column feedback_id: Unique feedback record identifier (PRIMARY KEY)
  - Column feedback_date: Date feedback was collected
  - Column household_key: Foreign key to household table
  - Column advisor_key: Foreign key to advisors table
  - Column feedback_text: Customer comments (max 200 characters)
  - Column satisfaction_score: Numeric satisfaction rating (0-100). Non-additive measure.


Relationships between Tables:
  public.account.advisor_key -> public.advisors.advisor_key
  public.account.household_key -> public.household.household_key
  public.account.business_line_key -> public.business_line.business_line_key
  public.fact_account_monthly.account_key -> public.account.account_key
  public.fact_revenue_monthly.account_key -> public.account.account_key
  public.transactions.product_id -> public.product.product_id
  public.fact_account_monthly.snapshot_date -> public.date.calendar_day
  public.fact_account_product_monthly.snapshot_date -> public.date.calendar_day
  public.fact_household_monthly.snapshot_date -> public.date.calendar_day
  public.fact_revenue_monthly.snapshot_date -> public.date.calendar_day
  public.transactions.transaction_date -> public.date.calendar_day
  public.fact_customer_feedback.feedback_date -> public.date.calendar_day


Important considerations about dates available:
  - Table public.fact_account_monthly, column snapshot_date: account dates between 2024-09-30 and 2025-09-30
  - Table public.fact_account_product_monthly, column snapshot_date: product account dates between 2024-09-30 and 2025-09-30
  - Table public.fact_household_monthly, column snapshot_date: household dates between 2024-09-30 and 2025-09-30
  - Table public.fact_revenue_monthly, column snapshot_date: revenue dates between 2024-09-30 and 2025-09-30
  - Table public.fact_customer_feedback, column feedback_date: feedback dates between 2024-01-01 and 2025-09-30

Key Terms:
  - Assets Under Management: Total market value of all client investments
    Use account_assets from fact_account_monthly table. 
         Don't aggregate over time as this measure is semi-additive. 
         You can aggregate this measure at advisor, household, or business line level.
  - Household: business or individual with a contractual agreement with advisors
    to get recent records, filter for household.household_status = 'Active' and household.to_date = '9999-12-31'

aum is synonym with Assets Under Management
client is synonym with Household
.
  
  *** Optimization Examples ***  
  
  A. Apply aggregation functions instead of returning a list of records.
      Example: - Analytical intent: "number of distinct ids in table where column equals 5"
               - Original sql query: "SELECT DISTINCT id FROM table WHERE column = 5;"
			         - Refined sql query: "SELECT COUNT(DISTINCT id) FROM table WHERE column = 5;"
			
  B. Group the data at a higher granularity.
     Example: If sql query shows data by days, group by months and return last N months.
  
  C. Group the data in buckets.
      Example: - Analytical intent: "Analyze the relationship between products.product_price and products.product_average-rating in the products table to determine if product price influences average rating."
               - Original sql query: "SELECT product_price, product_average_rating FROM products GROUP BY product_price, product_average_rating"
			         - Refined sql query: "SELECT 
                                         CASE 
                                             WHEN product_price < 10 THEN '<$10'
                                             WHEN product_price >= 10 AND product_price < 50 THEN '$10–$50'
                                             ELSE '$50+'
                                             END AS price_bucket,
                                         CASE 
                                             WHEN product_price < 10 THEN 1
                                             WHEN product_price >= 10 AND product_price < 50 THEN 2
                                             ELSE 3
                                             END AS price_bucket_sort,                                             
                                         ROUND(AVG(product_average_rating), 2) AS avg_rating,
                                         COUNT(*) AS product_count
                                   FROM products
                                   GROUP BY price_bucket
                                   ORDER BY price_bucket_sort;"
   
  
  D. Apply filters.
     Examples: - Time-based filters: Show records for the last 3 months. Check the database schema for date range information under "Important considerations about dates available".
              - Filter for a single company. Use values from the database schema. 
  
  E. Show top records.
     Provide a snapshot of data by retrieving maximum 20 rows and 5 columns.
     Example: The Analytical intent is: "average sale per customer", but there are too many customers, so show top N.
                       - Original sql query: "SELECT customer_name, avg(sale) as avg_sale from sales group by customer_name"
			                 - Refined sql query: "SELECT customer_name, avg(sale) as avg_sale from sales group by customer_name ORDER BY avg_sale desc limit 10;"

  *** Optimization Guidelines ***  
      1. Do not eliminate key dimensions that are explicitly part of the analytical intent.  
         For example, if the user asks for time based analysis per customer, do not drop time or customer attributes. 
         Instead, you can use optimization example D (filter date range) or example B (aggregate time at higher level).
'''

test_state = {
'objects_documentation':objects_documentation,
'sql_dialect': sql_dialect,
'messages_log':[],
'intermediate_steps' : [],
'analytical_intent': analytical_intent,
'current_question':question,
'current_sql_queries': [{'explanation':'','insight':'','query':query,'result':''}],
'generate_answer_details': {
    'key_assumptions': [],
    'agent_questions': [],
    'ambiguity_explanation': ''
},
'llm_answer': AIMessage(content=''),
'scenario': ''
,'search_terms_output':search_terms_output
}

In [ ]:
from agent import execute_sql_query
execute_sql_query(test_state)

In [10]:
test_state['current_sql_queries']

[{'explanation': '',
  'insight': 'As of September 30, 2025, there are 475 active households with a combined total of approximately $128.8 million in account assets. This represents an average of about $271,000 in assets per household.',
  'query': "SELECT SUM(fam.account_assets) AS total_account_assets, COUNT(DISTINCT fam.household_key) AS household_count FROM public.fact_account_monthly fam INNER JOIN public.household h ON fam.household_key = h.household_key WHERE fam.snapshot_date = '2025-09-30' AND h.household_status = 'Active' AND h.to_date = '9999-12-31'",
  'result': '128765983.83 475'}]